In [1]:
import pandas as pd
import numpy as np
from ckiptagger import WS, POS, NER
import re

In [2]:
# ETL1
df =pd.read_excel('200801to202008falutTypes.xlsx')
colNameZh = [_.split('\n')[0] for _ in df.columns]
colNameEng = [_.split('\n')[1] for _ in df.columns]
df.columns = colNameEng

# filter NA
df = df[df['CE_Comment'].notna()] # 缺失內容非空 (22483, 9)
df = df[ df['CE_Comment'].apply(lambda x: pd.to_numeric(x, errors='coerce')).isna() ] # 缺失內容非單一數字 (22481, 9)
df = df[df['CE_Item2'].notna()] # 條非空 (15198, 9)
df.to_csv('data_ETL1.csv',index=False)
del df

In [3]:
# ETL2-1: retrive punctuation
import re
import pickle
df = pd.read_csv('data_ETL1.csv')

punctuation = list()
for i, s in enumerate(df['CE_Comment']):
    s = re.findall(r'[^\u4e00-\u9fff]+', s)
    s = ''.join(s)
    s = re.findall(r'[^\w]+', s)
    s = ''.join(s)
    punctuation = punctuation + list(s)
else:
    punctuation = set(punctuation)
    pickle.dump(punctuation, open('punctuation.pkl', 'wb'))

In [4]:
# ETL2-2: tokenized by ckip, delimiter by puncuation
from ckiptagger import WS, POS, NER
from ckiptagger import data_utils, construct_dictionary
path = "./ckip"
ws = WS(path)
# pos = POS(path)
# ner = NER(path)

df = pd.read_csv('data_ETL1.csv')
punctuation = pickle.load(open('punctuation.pkl', 'rb'))
word_s = ws(df['CE_Comment'],
            sentence_segmentation=True,
            segment_delimiter_set=punctuation)

In [23]:
word_s1 = [ [_ for _ in w if _ not in punctuation]   for w in word_s ]
df['token'] =['@'.join(_) for _ in word_s1]
df.to_csv('data_ETL2noPunc.csv', index=False)

In [26]:
df

,CI_NobsKey,CE_CI_Block,CE_Item1,Ex_Tittle,CE_Item2,CE_Item3,CE_Item4,CE_Item5,CE_Comment,token
0,200800001,1,250,其他非屬採購法相關法令之缺失,2,NaN,NaN,NaN,(1)\t依政府採購法第30條第1項第1款勞務採購，得免收押標金、保證金之規定。本案押標金1...,1@依@政府@採購法@第30@條@第1@項@第1@款@勞務@採購@得@免@收押@標金@保證金...
1,200801001,8,250,其他非屬採購法相關法令之缺失,6,1,NaN,NaN,評選總表未載明受評廠商標價、本委員會全部委員姓名、職業等，未符「採購評選委員會審議規則」第6...,評選@總表@未@載明@受@評@廠商@標價@本@委員會@全部@委員@姓名@職業@等@未@符@採...
2,200801001,8,250,其他非屬採購法相關法令之缺失,3,NaN,NaN,NaN,工作小組初審意見應依「採購評選委員會審議規則」第3條規定載明工作小組姓名、職稱及專長、受評廠...,工作@小組@初審@意見@應@依@採購@評選@委員會@審議@規則@第3@條@規定@載明@工作@...
3,200801001,1,251,政府採購法施行細則,6,NaN,NaN,NaN,經審視本案於96年6月26日公開招標公告資料中所刊載之採購金額級距為「公告金額以上未達查核金...,經@審視@本@案@於@96年@6月@26日@公開@招標@公告@資料@中@所@刊載@之@採購@...
4,200801002,3,251,政府採購法施行細則,53,NaN,NaN,NaN,(2)\t底價表內「預計金額」應依採購法施行細則第53條及本府96年8月27日府授工採字第0...,2@底價表@內@預計@金額@應@依@採購法@施行@細則@第53@條@及@本府@96年@8月@...
...,...,...,...,...,...,...,...,...,...,...
15193,202007255,2,36,採購評選委員會審議規則,6,NaN,2,NaN,項次28-採購評選委員會審議規則第6條第2項規定：「不同委員之評選結果有明顯差異時，召集人應...,項次@28@採購@評選@委員會@審議@規則@第6@條@第2@項@規定@不同@委員@之@評選@...
15194,202007266,1,251,政府採購法施行細則,43,NaN,NaN,NaN,【第4項】本採購案招標，於109年2月6日辦理更正公告，截止投標日期為109年2月12日，惟...,第4@項@本@採購案@招標@於@109年@2月@6日@辦理@更正@公告@截止@投標@日期@為...
15195,202008303,1,15,中央機關未達公告金額採購招標辦法,2,NaN,1,3,【項次8】依中央機關未達公告金額採購招標辦法第2條第1項第3款規定略以：「……公開於主管機關...,項次@8@依@中央@機關@未@達@公告@金額@採購@招標@辦法@第2@條@第1@項@第3@款...
15196,202008303,1,251,政府採購法施行細則,6,NaN,1,3,【項次1】依政府採購法施行細則第6條第1項第3款規定：「招標文件含有選購或後續擴充項目者，應...,項次@1@依@政府@採購法@施行@細則@第6@條@第1@項@第3@款@規定@招標@文件@含有...


['(',
 '1',
 ')',
 '\t',
 '依',
 '政府',
 '採購法',
 '第30',
 '條',
 '第1',
 '項',
 '第1',
 '款',
 '勞務',
 '採購',
 '，',
 '得',
 '免',
 '收押',
 '標金',
 '、',
 '保證金',
 '之',
 '規定',
 '。',
 '本案',
 '押標金',
 '10萬',
 '元',
 '、',
 '履約',
 '保證金',
 '20萬',
 '元',
 '、',
 '差額',
 '保證金',
 '78萬餘',
 '元',
 '，',
 '得標',
 '廠商',
 '同',
 '時間',
 '繳納',
 '約',
 '98萬餘',
 '元',
 '，',
 '履約',
 '65',
 '個',
 '日曆天',
 '全',
 '案',
 '完成',
 '履約',
 '驗收',
 '合格',
 '一',
 '次',
 '付款',
 '，',
 '廠商',
 '始',
 '可',
 '請領',
 '價金',
 '為',
 '120萬餘',
 '元',
 '。',
 '又',
 '契約',
 '驗收',
 '分期',
 '中',
 '及',
 '期末',
 '二',
 '次',
 '，',
 '建議',
 '繳納',
 '押標金',
 '、',
 '履約',
 '保證金',
 '之',
 '必要性',
 '及',
 '檢討',
 '契約',
 '價金',
 '分期',
 '給付',
 '及',
 '退還',
 '保證金',
 '之',
 '可行性',
 '，',
 '以',
 '減輕',
 '廠商',
 '資金',
 '調度',
 '壓力',
 '。']